GLF Kernel - Tutorial
===

#### Installation
See the instructions in the GitHub repository ([https://github.com/kwarc/glf_kernel](https://github.com/kwarc/glf_kernel)).

#### Very Brief Intro: Jupyter Cells
In Jupyter, there are two types of cells:
* *markdown cells* (such as this one), which can be used for documentation and explanations
* *code cells*, which contain GF grammars and shell commands if you use the GF kernel

You can edit a cell by clicking on it (double clicking in the case of markdown cells).
To execute the cell, you can press <kbd>Ctrl</kbd>+<kbd>Return</kbd> or <kbd>Shift</kbd>+<kbd>Return</kbd>. If you want to execute all cells in the notebook, you can choose *Cell* -> *Run All* in the top menu.

## GLF Kernel Commands

You can use this kernel to create new MMT archives or switch to existing ones using the `archive` command.

`archive 'archive-name'` will set your current working archive to the speciified archive or create a new archive if it doesn't exist already.

The default archive for this kernel will be `COMMA/JUPYTER`

In [1]:
archive comma/jupyter

Changed to archive comma/jupyter

If you wish to save your MMT and GF source files into a specific subdirectory, you can do this with the `subdir` command.
Executing `subdir` without arguments will return your current subdirectory.
Note that the subdirectory you specify is always relative to the current archives source directory. 
So if you'd like to create a nested subdirectory you can do this with something like:

In [2]:
subdir test/test1

Changed to subdirectory test/test1

You can also use the archive command to get an overview of your current archive. Let's see how it looks like:

In [3]:
archive

comma/jupyter
└── source
   └── test
       └── test1
           ├── Grammar.gf
           └── GrammarEng.gf


## Defining Grammars
You can define GF Grammars directly in the notebook using code cells.

These grammars will automatically be saved into the source (or when specified sub-) directory of the current archive.

For toggling line numbers in a cell, you need to switch to command mode (<kbd>Esc</kbd>) and press <kbd>L</kbd>.

In [4]:
abstract Grammar = {
    cat
        Person ; Action ; Sentence ;

    fun
        john, mary : Person ;
        run, be_happy : Action ;
        make_sentence : Person -> Action -> Sentence ;
        and : Sentence -> Sentence -> Sentence ;
}

Defined Grammar

In [5]:
concrete GrammarEng of Grammar = {
    lincat
        Person = Str ;
        Action = Str ;
        Sentence = Str ;
    lin
        john = "John" ;
        mary = "Mary" ;
        run = "runs" ;
        be_happy = "is happy" ;
        make_sentence person action = person ++ action ;
        and a b = a ++ "and" ++ b ;
}

You can also run normal GF commands, as you are used to:

In [6]:
parse -cat=Sentence "John runs"

make_sentence john run

For displaying graphs in the notebook, you need to pipe them into the `show` command.

In [7]:
parse -cat=Sentence "John runs and Mary is happy" | view_tree | show

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x88\x00\x00\x00\xfb\x08\x06\x00\x00\x00\xe9`e\xf…

## MMT For Target Logic and Semantics Construction

You can also use it to define new theories and views that you would like to use with your grammars.
Every time a new theory, view or grammar is specified MMT will build it.

The namespace of defined theories or views will automatically be set to the current archive.

In [8]:
theory FolLogic : ur:?LF =
	o : type ❙
	ι : type ❙
	and : o ⟶ o ⟶ o ❘ # 1 ∧ 2 ❙
❚

Created theory FolLogic

In [9]:
theory DomainTheory : ?FolLogic =
	john : ι ❙
	mary : ι ❙
	run : ι ⟶ o ❙
	happy : ι ⟶ o ❙
❚

Created theory DomainTheory

In [10]:
view grammarSem : jupyter/Grammar.gf?Grammar -> ?DomainTheory =
	Person = ι ❙
	Action = ι ⟶ o ❙
	Sentence = o ❙
	john = john ❙
	mary = mary ❙
	run = run ❙
	be_happy = happy ❙
	make_sentence = [p,a] (a p) ❙
	and = [a,b] b ∧ a ❙
❚

Created view grammarSem

Now you can send semantics construction requests to the GLF-Server using the `construct` command.
The `construct` command takes an AST as argument, e.g.:

In [11]:
construct and (make_sentence john run) (make_sentence mary be_happy)

(happy mary)∧(run john)

`construct` always uses the last view you specified in the notebook.
Should you wish to change the view that is used for the requests you can do it via the -v option.
This also sets the specified view as the default view for further requests.
You can also use `construct` in a pipe-like way like most other GF commands:

In [12]:
parse -cat=Sentence "John runs and Mary is happy" | construct -v grammarSem

(happy mary)∧(run john)

## Further Kernel Commands

Apart from `export` and `view`, there are two more kernel commands: `clean` and `help`.

`clean` removes `.gfo`, `.dot` and `.png` files.

`help` displays information on the kernel commands (not the GF shell commands).
If you want information on the GF shell commands instead, you should use
the shortcut `h` instead:

In [13]:
h reload

r, reload

repeat the latest import command

In [14]:
help

Available kernel commands:

show 'graph' : shows the graph in the notebook

clean : remove all .dot, .png and .gfo files from the current directory

export 'name' : export the grammar 'name' to your current directory

archive [-c] 'MMT-archive' : sets 'MMT-archive' as the current archive. The -c option creates the archive if it does not exist yet

...

To display more information on the GF shell commands use 'h'

All kernel commands are pipeable in a GF-shell like manner using '|', e.g. parse 'Sentence' | vt | view

You can use the kernel as an editor for your grammars, views or theories.

Stated grammars are automatically imported upon definiton.

Whenever a grammar, theory or view is defined the current MMT archive will be built